# Ceneo Scraper

## Lista kroków "manualnych"

1. Wejdź na stronę produktu / z opiniami o produkcie
2. Dla każdej opinii na stronie
    1. Skpiuj opinię
    2. Wklej opinię do edytora tekstu
3. Przejź do klejnej strony z opiami
4. Powtarzaj kroki 2-3 dopóki są strony z opiniami  

## Lista kroków scrapera

1. Wysłanie żądania dostępu do zasobu - strona z opiniami o produkcie
2. Wydobycie z kodu HTML fragmentów odpowiadających opiniom
3. Dla każdej opinii na stronie
    1. Wydobycie z kodu opini jej składowych 
    2. Dodanie do słownika reprezntującą pojedynczą opinię do listy
4. Przejdź do kolejnej strony z opiniami
5. Powtarzaj kroki 1-4 dopóki są listy z opiniami
6. Zapis listy słowników do pliku JSON

## Struktura opini w serwisie Ceneo.pl

|składowa|nazwa|selektor|
|--------|-----|--------|
|identyfikator opinii|opinion_id||
|autor|author||
|rekomendacja|recommendation||
|treść opinii|content||
|lista zalet|pros||
|lista wad|cons||
|data wystawienia opinii|post_date||
|data zakupu produktu|purchase_date||
|ile osób uznało opinię za przydatną|useful||
|ile osób uznało opinię za nieprzydatną|useless||

Produkty:
1. 63717975
2. 124893467
3. 108481121
4. 114228736
5. 28020568

In [201]:
import requests, os, json
from bs4 import BeautifulSoup

In [202]:
product_code = input("Podaj kod produktu z Ceneo.pl: ")
url = f"https://www.ceneo.pl/{product_code}#tab=reviews"

In [203]:
def get_data(ancestor, selector, attribute=None, return_list=False):
     if return_list:
          return [tag.text.strip() for tag in ancestor.select(selector)] 
     if attribute:
          if selector:
               try:
                    return ancestor.select_one(selector)[attribute].strip()
               except TypeError:
                    return None
          return ancestor[attribute].strip()
     try:
          return ancestor.select_one(selector).text.strip()
     except AttributeError:
          return None

In [204]:
selectors = {
            'opinion_id':(None, "data-entry-id"),
            'author': ("span.user-post__author-name",),
            'recommendation': ("span.user-post__author-name",),
            'stars': ("span.user-post__score-count",),
            'content': ("div.user-post__text",),
            'pros': ("div.review-feature__title--positives ~ div.review-feature__item", None, True),
            'cons': ("div.review-feature__title--negatives ~ div.review-feature__item", None, True),
            'post_date': ("span.user-post__published > time:nth-child(1)", "datetime"),
            'purchase_date': ("span.user-post__published > time:nth-child(2)", "datetime"),
            'useful': ("button.vote-yes > span",),
            'useless': ("button.vote-no > span",)
        }

In [205]:
all_opinions=[]
while(url):
    print(url)
    response = requests.get(url)
    page = BeautifulSoup(response.text, "html.parser")
    opinions = page.select("div.js_product-review") 
    for opinion in opinions:
        single_opinion = {
            key: get_data(opinion, *value)
                for key, value in selectors.items()
        }
    all_opinions.append(single_opinion)
    try:
        url = "https://ceneo.pl"+page.select_one("a.pagination__next")["href"]
    except TypeError:
        url = None

https://www.ceneo.pl/28020568#tab=reviews
https://ceneo.pl/28020568/opinie-2
https://ceneo.pl/28020568/opinie-3
https://ceneo.pl/28020568/opinie-4
https://ceneo.pl/28020568/opinie-5


In [206]:

if not os.path.exists("opinions"):
    os.mkdir("opinions")
jf = open(f"opinions/{product_code}.json","w",encoding="UTF-8")
json.dump(all_opinions, jf, indent=4, ensure_ascii=False)
jf.close()